In [ ]:
## IMPORTANT: On Colab, we expect your homework to be in the cs189 folder
## Please contact staff if you encounter any problems with installing dependencies
import sys
IS_COLAB = 'google.colab' in sys.modules
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/cs189/hw/hw2
    %pip install -r ./requirements.txt
    !pip install -U kaleido plotly
    import kaleido
    kaleido.get_chrome_sync()

import plotly.io as pio
pio.renderers.default = pio.renderers.default + "+png"


In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("arena_warmup.ipynb")

<link rel="stylesheet" href="berkeley.css">

<h1 class="cal cal-h1">Homework 02 – Welcome to the Arena (Warmup)</h1>

CS 189, Fall 2025

In this homework you will get more experience with logistic regression in two very different settings: creating leaderboards and predicting model responses.

We will be taking real data from [LMArena](https://lmarena.ai/), a popular platform for crowsourcing evaluations of large language models and recreating their leaderboards, with a few fun extra steps along the way.

The chats can be viewed interactively by accessing [ChatBot-Arena-Viewer](https://huggingface.co/spaces/BerkeleyML/Chatbot-Arena-Viewer) through hugging face. Much of the first half of this homework was first written by Prof Gonzalez back when his students first started the project, and now LMArena is a standard evaluation for large language models and turned into a company! Don't let anyone tell you logistic regression isn't valuable, it's worth at least $600 Million.

---

## Due Date: Saturday, October 4, 11:59 PM

This assignment is due on **Saturday, October 4, at 11:59 PM**. You must submit your work to Gradescope by this deadline. Please refer to the syllabus for the [Slip Day policy](https://eecs189.org/fa25/syllabus/#slip-days). No late submissions will be accepted beyond the details outlined in the Slip Day policy.

### Submission Tips
- **Plan ahead**: We strongly encourage you to submit your work several hours before the deadline. This will give you ample time to address any submission issues.
- **Reach out for help early**: If you encounter difficulties, contact course staff well before the deadline. While we are happy to assist with submission issues, we cannot guarantee responses to last-minute requests.

<!-- ---

### Assignment Overview

This notebook contains a series of tasks designed to help you practice and apply concept you learned in class to a very practical task - Ranking LLM Models. You will complete all the TODOs in the notebook, which include both coding and written response questions. Some tasks are open-ended, which allows you to explore and experiment with different approaches.

Question 1: Explore the data

Question 2 : Analyze the prompts and their effects

Question 3: Explore model ranking and win rates

Question 4: Think about learning model strengths

Question 5: Generate confidence intervals

--- -->

### Key Learning Objectives

1. Learn how to evaluate large language models (LLMs) using pairwise comparison data from LMArena
2. Analyze battle distributions and compute win rates
3. Find subsets of prompts which result in changes to the leaderboard
    
---

### Collaboration Policy
You are encouraged to discuss high-level concepts with your peers. However:
- All submitted work must be written in your own words and code.
- Do not share or copy solutions directly.
- List any collaborators (students you worked with) in the line below. Include their name and SID:

**Your Collaborators**: **TODO**

### AI Tools Usage Disclosure
We allow the use of AI tools (e.g., ChatGPT, Copilot) **only as support**, not as a replacement for your own reasoning. To ensure transparency, you must acknowledge any use of AI tools.

Please answer with one of the following options:
- **A) I did not use any AI tools for this homework.**
- **B) I used AI tools in the following way(s):**  
  (describe briefly, e.g., “Used ChatGPT to get hints for debugging a NumPy indexing error”)


**Your Answer**: **TODO**
    
---

### Grading Breakdown

<!-- <div align="center"> -->


| Question  | Manual Grading? | Points |
| --------- | --------------- | ------ |
| q1a       | No              | 1      |
| q1b       | No              | 1      |
| q1c       | Yes             | 2      |
| q2a       | No              | 2      |
| q2b       | Yes             | 1      |
| q2c       | Yes             | 3      |
| q3a       | Yes             | 2      |
| q3b       | No              | 2      |
| q3c       | Yes             | 5      |
| q3d       | Yes             | 2      |
| q3e       | Yes             | 1      |
| q3eFRQ    | Yes             | 3      |
| **Total** |                 | **26** |


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"
#set fixed seed of 189
np.random.seed(189)

<div style="display: flex; align-items: flex-start;">
  <div style="flex: 1; padding-right: 24px;">
    <img src="https://i.imgur.com/tbrkWVX.png" alt="LMArena" style="width: 100%; max-width: 350px; display: block; margin: 0 auto;">
  </div>
  <div style="flex: 2;">
    <h1 style="font-size: 2em; font-weight: bold; margin-bottom: 0.5em;">What is LMArena?</h1>
  </div>
</div>

LMArena (previously known as Chatbot Arena) is a platform that evaluates generative models (e.g. chatbots or image generation models) through anonymous, crowd-sourced pairwise comparisons. Users enter prompts for two anonymous models to respond to and vote on the model that gave the better response, in which the model's identities are revealed (shown below). Users can also choose models to test themselves, but for the purposes of this homework we will only focus on the anonymous side-by-side comparisons, which we call **"battles"** -  since those are what are used to calcuate the leaderboard.

<div style="text-align: center;">
  <img src="https://i.imgur.com/rgv0jCb.png" alt="LMArena" style="display: block; margin-left: auto; margin-right: auto; width: 60%;">
</div>


In this homework we will investigate what these battles look like, how we can use these pairwise comparisons to get a leaderboard, and how we can find certain features of model responses that have an influence on preference.

Although it is not required for this notebook, the [Chatbot Arena paper](https://arxiv.org/abs/2403.04132) can provide good intuition on how to answer the free response questions. 

### Download Data

First, let's load a set of publicly released arena battles from [Hugging Face](https://huggingface.co/datasets/lmarena-ai/arena-human-preference-100k) — a popular website for sharing machine learning datasets and models. They also have a ton of great visualization tools with [Gradio](https://www.gradio.app/docs), which you will also get some experience with in this homework.


**Note**: Before you get started with the homework, we reccomend you make a [huggingface account](https://huggingface.co/welcome) to play around with creating or modifying our data visualization apps! It is also a great general hub for downloading the majority of popular datasets in machine learning. Once you make the account, generate the token for login [huggingface token]
(https://huggingface.co/docs/hub/security-tokens). This should be similar to how you generate a token for your git account.

In [ ]:
# To log into huggingface, uncomment the line below and re-run the cell

#! pip install ipywidgets
#from huggingface_hub import notebook_login
#notebook_login()

In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset (this will take a few minutes to download)
# if you don't have an account it may throw a warning to create a HF_TOKEN but you should still have access to the dataset if you skip this
ds = load_dataset("lmarena-ai/arena-human-preference-100k")
battles = ds['train'].to_pandas()

<link rel="stylesheet" href="berkeley.css">

Now let's look at the format of this data.

Printing out the first row we see there are many fields with the most important being:

<ul class="cal cal-list">
  <li><strong>question_id (str)</strong> - the ID of that battle</li>
  <li><strong>model_a, model_b (str)</strong> - the models which participated in this battle</li>
  <li><strong>winner (str)</strong> - which response the user preferred: can be <span class="cal-code-inline">model_a</span>, <span class="cal-code-inline">model_b</span>, <span class="cal-code-inline">tie</span>, and <span class="cal-code-inline">tie (bothbad)</span></li>
  <li><strong>conversation_a (dict)</strong> - the conversation between the user and model a</li>
  <li><strong>conversation_b (dict)</strong> - the conversation between the user and model b (note that the user turns in conversation_a and conversation_b are the same since this is a side by side comparison)</li>
  <li><strong>turn (int)</strong> - number of turns in the conversation (1 turn means the user asked 1 question, 2 turns means the user asked a question, got an answer, then asked another question, got the response, then voted)</li>
  <li><strong>language (str)</strong> - the language of the user prompt</li>
</ul>

We also have some other columns that may be useful for us later:

<ul class="cal cal-list">
  <li><strong>is_code (bool)</strong> - either the prompt, the response, or both contains code</li>
  <li><strong>is_refusal (bool)</strong> - one of the models refused to answer (usually this is because the model thinks it would be unethical to answer)</li>
  <li><strong>dedup_tag (dict)</strong> - indicates whether the prompt appears very often (high_frequency) and if it does, whether it will be sampled (subsampled). We subsample these high frequency prompts so that common questions don't overly influence the leaderboard.</li>
  <li><strong>category_tag (dict)</strong> - tags for question type (e.g. math and instruction following). These are assigned via an LLM labeler, more details on what the categories are in this <a href="https://blog.lmarena.ai/blog/2024/arena-category/">blog post</a> (the criteria tags correspond to the hard prompts category described in the blog post).</li>
</ul>

In [ ]:
print("Columns:\n*", "\n* ".join(battles.columns))

In [ ]:
battles.head(5)

In [ ]:
example = battles.iloc[4]
print(f"Conversation A (model = {example['model_a']}):")
print(example['conversation_a'])
print(f"Conversation B (model = {example['model_b']}):")
print(example['conversation_b'])
print("Category Tag:")
print(example['category_tag'])

### Exploratory Analysis

Before we get into leaderboard calculation, let's first conduct some basic exploratory analysis to highlight a few key properties and caveates with this data.

In [ ]:
battles.winner.hist(title="Counts of Battle Outcomes", text_auto=True)

#### NOTE: Notice how LMArena has two types of ties: tie (both bad) and just tie.

### Battle Counts

We see that certain models participate in more battles. This is due to two reasons:
1. Several different matching and sampling algorithms were used. LMArena employs weighted sampling methods, which assign greater weights to better models.
2. Since models are added to the arena when they come out, some models have been on the arena for many months while others have only been on for a few weeks.

In [ ]:
fig = pd.concat([battles["model_a"], battles["model_b"]]).value_counts().plot.bar(title="Battle Count for Each Model", text_auto=True)
fig.update_layout(xaxis_title="Model", yaxis_title="Battle Count", height=400, showlegend=False)
fig

# **Question 1: Data Exploration**

## **Question 1a**
Since it can be hard to reason over that many models, we want to look at the top 20 models by battle count.

**Task:** 
Return the a list of top 20 models by battle count for both `model_a` and `model_b` combined, SORTED by the battle count.

In [ ]:
#TODO
selected_models = ...

In [ ]:
grader.check("q1a")

## **Question 1b**

**Task:** 
Now, let's filter out and select the battles that are between the top 20 models we got from question 1a. Fill in the `subselect_battles` function to return the battles dataframe contatining only the selected models and the battles dataframe containing the selected models with ties removed. 

**Hint:** You may find it helpful to use a boolean array.

In [ ]:
from typing import Tuple, Set
import pandas as pd

def subselect_battles(
    battles: pd.DataFrame, 
    selected_models: Set[str]
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    # Filters the battles DataFrame to only include battles between the selected models.
    # Returns a tuple of the dataframe filtered by models and the dataframe filtered my models with ties removed.
    ...
    return selected_battles, selected_battles_no_ties
  
selected_battles, selected_battles_no_ties = subselect_battles(battles, selected_models)

In [ ]:
grader.check("q1b")

We’re going to visualize how often each pair of models battled each other, similarly to the heatmap on the right side of **Figure 2** in the [LMArena paper](https://arxiv.org/pdf/2403.04132). We’ll sort models by their total number of battles and show the top **N** (default 30). Each cell of the heatmap shows the battle counts for the *(Model A, Model B)* match-up.

In [ ]:
def visualize_battle_count(battles, title, show_num_models=30):
    """
    Input:
        battles : pd.DataFrame with columns ['model_a','model_b', ...]
        title   : str, title for the plot
        show_num_models : int, how many top models (by total battle count) to display

    Output:
        fig : plotly.graph_objects.Figure heatmap of symmetric battle counts
    """
    ptbl = pd.pivot_table(battles, index="model_a", columns="model_b", aggfunc="size", fill_value=0)
    battle_counts = ptbl + ptbl.T
    ordering = battle_counts.sum().sort_values(ascending=False).index
    ordering = ordering[:show_num_models]

    fig = px.imshow(
        battle_counts.loc[ordering, ordering],
        title=title,
        text_auto=True
    )
    fig.update_layout(
        xaxis_title="Model B",
        yaxis_title="Model A",
        xaxis_side="top",
        height=1000,
        width=1000,
        title_y=0.07,
        title_x=0.5,
        font=dict(size=10)
    )
    fig.update_traces(
        hovertemplate="Model A: %{y}<br>Model B: %{x}<br>Count: %{z}<extra></extra>"
    )
    return fig

Let's visualize selected_battles and selected_battles_no_ties subsequently.

### Visualize All Selected Battles

In [ ]:
visualize_battle_count(selected_battles, title="Battle Count of Each Combination of Models", show_num_models=30)

### Visualize Selected Battles No Ties

In [ ]:
visualize_battle_count(selected_battles_no_ties, "Battle Count for Each Combination of Models (without Ties)")

<!-- BEGIN QUESTION -->

## **Question 1c**
We see many battles between top models (e.g., Claude, GPT, Gemini), while smaller models (e.g., Llama-3-8B) have fewer battles. This is because LMArena employs weighted sampling methods, which assign greater weights to better models.

**Task:** Answer the question below (in the cell after the question cell).

```otter
Q: Why might LMArena pair strong models vs. strong models more often than strong vs. smaller models? 
Think about statistical power, how quickly you can differentiate models, and ranking uncertainty among top-tier systems.
```

```otter
YOUR ANSWER:
```

<!-- END QUESTION -->

## Languge Distribution
Let’s explore which languages appear in LMArena battles. See how many language [LMArena](https://lmarena.ai/) is being used for! This is similar to the heatmap on the right side of **Figure 10** in the [LMArena paper](https://arxiv.org/pdf/2403.04132).

The full distribution gives context, but the range across languages can be huge. Try zooming in by highlighting the region of the plot you want to view.

In [ ]:
lang_counts_all = battles["language"].value_counts()

fig_lang_all = px.bar(
    lang_counts_all,
    title="Distribution of Languages",
    text_auto=True,
    height=400
)
fig_lang_all.update_layout(
    xaxis_title="Language",
    yaxis_title="Count",
    showlegend=False
)
fig_lang_all.show()

## Number of Conversation Turns

Now let's also try to explore conversation turns.

In [ ]:
fig = px.histogram(battles["turn"],
             title=f"Number of Conversation Turns",
             text_auto=True, height=400, log_y = True)
fig.update_layout(xaxis_title="Turns", yaxis_title="Count", showlegend=False)
fig.update_traces(marker_line_color='black', marker_line_width=1)
fig

## Brief Introduction to Gradio

#### Visualizing and Sharing Your Clusters

Now we have plotted a lot of stats but what does this data actually *look* like? You can try printing out conversations in your notebook but reading large amounts of text can be a pain, so how can we better view our data? 

There are many different ways you can do this but we will show you one UI package which is fast to vibe code and sharable so you can show all these wonderful LLM conversations with your friends (or your future advisors and managers). 

#### Example: Gradio App

One highly recommended tool for this purpose is **Gradio**. Gradio is a Python library for quickly building interactive web apps for your models and visualizations. These apps are easy to share—even with people who don’t have Python installed!

- Gradio integrates with [Hugging Face Spaces](https://huggingface.co/spaces), a popular website for hosting and sharing machine learning demos, datasets, and models.

One of our staff members, Lisa, **created an [app to go through random conversations](https://huggingface.co/spaces/BerkeleyML/Chatbot-Arena-Viewer) in the Chatbot Arena dataset.** She was able to code this up within 5 minutes (this is an opportunity for you to leverage LLMs to achieve useful tools). Also fun fact: LMArena was a gradio app up until a few months ago!

<div style="text-align: center;">
  <img src="https://i.imgur.com/VvGs5nR.png" alt="GradioConversation" style="display: block; margin-left: auto; margin-right: auto; width: 60%;">
</div>

Feel free to play around with Gradio experience to view raw chatbot arena conversations or for the cluster visualization you will do in a later problem in this homework.

[Gradio documentation](https://www.gradio.app/docs) – reference for building your own apps. To publicly share your app, you may need to sign up for a free [Hugging Face account](https://huggingface.co/welcome) and accept their terms, and create a token for the app linked with your account. 

---

# **Question 2: Model Rankings**

Now that we have explored our data, let's consider how to use these pairwise battles to rank the models by preference. Our goal is to assign a "strength" parameter to each model that quantifies how likely it is to win against others.

Given we analyzed $M=20$ models and $N=40k$ battles ($26k$ excluding ties), we want to estimate a skill parameter $S_m$ for each model $m \in \{1, \ldots, M\}$. This parameter $S_m$ should reflect the overall ability of model $m$ to be preferred over other models.

Before we move on to more sophisticated probabilistic models that estimate these strength parameters, let’s build some intuition by starting with a simpler metric: the average win rate.

Average win rate is simple: a model's average win rate is the proportion of battles they competed in which resulted in them winning. 

##  **Question 2a**

LMArena defines the win rate for a model as the average fraction of times it defeats another model across all its match-ups. 

**Task:**
Implement the `compute_pairwise_win_fraction` function, which:

1. Calculates the fraction of times each model beats each other model across all battles.

2. Returns a square DataFrame where entry (i, j) is the fraction of times model i beats model j. The colmns should be the select model names as well as the index (similar to a confusion matrix). Any model pairings which do not have any battles should be given a NaN value. For instance, diagonal of `row_beats_col` should be NaN as no battles exist between a model and itself.

3. The rows and columns of your `row_beats_col` dataframe should be ordered by their average win rate against all other models (i.e. order from strongest to weakest models)

Tips:
* Do not use your `selected_models` variable in your function, otherwise you may run into autograder issues. Instead define a variable which is the list of models in you input `battles` dataframe.

In [ ]:
def compute_pairwise_win_fraction(battles):
    #TODO

    return row_beats_col

In [ ]:
grader.check("q2a")

<!-- BEGIN QUESTION -->

##  **Question 2b**


Let’s visualize how often **Model A** beats **Model B** in non-tied battles. Below we have used your `compute_pairwise_win_fraction` to create heatmap where each cell `(A, B)` displays the **fraction of A’s wins** over B. No TODOs or code to fill in here, this question allows us to viusally inspect your function in the Coding PDF assignment on gradescope. 

In [ ]:
def visualize_pairwise_win_fraction(battles, title):
    """
    Input:
        battles : pd.DataFrame of non-tied battles with ['model_a','model_b','winner', ...]
        title   : str
    Output:
        fig : plotly Figure heatmap (cell (A,B) = fraction A beats B)
    """
    row_beats_col = compute_pairwise_win_fraction(battles)
    fig = px.imshow(
        row_beats_col,
        color_continuous_scale='RdBu',
        text_auto=".2f",
        title=title
    )
    fig.update_layout(
        xaxis_title=" Model B: Loser",
        yaxis_title="Model A: Winner",
        xaxis_side="top",
        height=900,
        width=900,
        title_y=0.07,
        title_x=0.5
    )
    fig.update_traces(
        hovertemplate="Model A: %{y}<br>Model B: %{x}<br>Fraction of A Wins: %{z}<extra></extra>"
    )
    return fig


fig = visualize_pairwise_win_fraction(
    selected_battles_no_ties,
    title="Fraction of Model A Wins for All Non-tied A vs. B Battles"
)
fig


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

##  **Question 2c**


Now we can just average the win rate of model $i$ against all other models to get an estimate of strength.

In [ ]:
def get_pairwise_win_fraction_plot(battles, title=""):
    row_beats_col_freq = compute_pairwise_win_fraction(battles)
    pairwise_win_rate = row_beats_col_freq.mean(axis=1).reset_index()
    pairwise_win_rate.columns = ['model', 'win rate']

    # Rank (1 = best)
    pairwise_win_rate["rank"] = (
        pairwise_win_rate["win rate"].rank(ascending=False, method="dense").astype(int)
    )

    # Sort for plotting and freeze that order on the x-axis
    pairwise_win_rate = pairwise_win_rate.sort_values(by="win rate", ascending=False)
    model_order = pairwise_win_rate["model"].tolist()

    fig = px.bar(
        pairwise_win_rate,
        x="model",
        y="win rate",
        title=title,
        text="win rate",
        hover_data=["rank"]
    )
    fig.update_traces(texttemplate="%{text:.2f}", hovertemplate="<b>%{x}</b><br>win rate=%{y:.3f}<br>rank=%{customdata}")
    fig.update_layout(
        yaxis_title="Average Win Rate",
        xaxis_title="Model",
        showlegend=False
    )
    fig.update_xaxes(tickangle=45, categoryorder="array", categoryarray=model_order)
    return pairwise_win_rate, fig

pairwise_win_rate, fig = get_pairwise_win_fraction_plot(selected_battles_no_ties, title="Average Win Rate Against All Other Models (No Ties)")
fig.show()

Now that we’ve computed and visualized the average win rate of each model against all others, we can start reasoning about the results.
In the chart above, we see that some models have very similar average win rates. For example, some GPT, Claude, and Llama variants sit close together. On the other hand, smaller models like llama-3-8b and gemma-2-9b fall noticeably behind.

**Task: Answer the following questions in the cell below the question cell**

```otter
1. Identify at least two models that appear to have similar average win rates.
2. Compare their parameter sizes of each model (you can google their parameter counts, and if its a closed source model which does not list its parameter size assume its >100 billion parameters). Is there a relationship between parameter size and performance? Are there any models which stick out as unusally good or bad for its size?
3. Why might computing win-rate in this manner misrepresent model strength? Think about the distribution of battles per model pair and how an imbalance in model pairing counts could result in one model ranking high or lower than it should. 
```

```otter
YOUR ANSWER:
```

<!-- END QUESTION -->

# **Question 3: Prompt Analysis**

We have explored the general features of this dataset. When evaluating models, it’s also often useful to understand the types of questions that are asked. By grouping similar prompts together, we can analyze which models perform well on certain categories and poorly on others. This helps uncover biases in leaderboards (e.g., a model may excel at coding questions but struggle with creative writing).

**First, let's identify the most frequent prompts.**

In the code cell below, we've already done the following:
*   Extracted the first user message (prompt) from conversation_a
*   Filtered out only the battles in English and only kept the select models using your `subselect_battles` function from 1b

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD

def first_user_text(conv):
    return (conv[0].get("content") or "").strip()

battles['prompt'] = battles['conversation_a'].apply(first_user_text).fillna("")
eng_battles = battles[(battles['language'] == 'English') | (battles['language'] == 'unknown')]
eng_battles, eng_battles_no_ties = subselect_battles(eng_battles, selected_models)

Now, let's view the top 10 promiment prompts for battles in English.
Note that there is a caveat for LMArena data, where some of the lanauges are labeled as 'unknown'. This is the reason why we have `battles['language'] == 'unknown'` as part our filter.

In [ ]:
# Print the top 10 most common prompts along with their count and percentage of total prompts
top_prompts = eng_battles["prompt"].value_counts().head(10)
for i, (prompt, count) in enumerate(top_prompts.items(), 1):
    print(f"Rank {i}: {count} samples ({round(100 * count/len(eng_battles), 2)}%)\n{prompt}\n")

# print the total percentage of prompts that are 1 of the top 10 prompts
top_10_percentage = sum(top_prompts) / len(eng_battles)
print(f"Total percentage of prompts that are 1 of the top 10 prompts: {round(100 * top_10_percentage, 2)}%")

<!-- BEGIN QUESTION -->

## **Question 3a**

When evaluating models, it’s useful to understand which prompt types are over-represented and how these popular prompts can influence the leaderboard. Below we have plotted the leaderboard when all of the top 10 prompts are removed. 

**Task:**  Compare the leaderboard generated from `get_pairwise_win_fraction_plot(eng_no_top_prompts_battles)` to the leaderboard you generated in the previous problem. In 2-3 sentences, answer the following:

```otter
1. Is there a difference between the two leaderabords? List any models in the top 5 that have changed position
2. What is a potential reason that this leaderboard has shifted? We are looking for an answer which relates to potential model behavior, not just "prompts were removed thus the leaderboard changed"
```

In [ ]:
# remove top prompts top_prompts from eng_battles_no_ties
eng_battles_no_ties_no_top_prompts = eng_battles_no_ties[~eng_battles_no_ties["prompt"].isin(top_prompts.index)]

pairwise_win_rate, fig = get_pairwise_win_fraction_plot(eng_battles_no_ties_no_top_prompts, title="Average Win Rate Against All Other Models (No Top Prompts)")
fig.show()

```otter
YOUR ANSWER:
```

<!-- END QUESTION -->

## **Question 3b**
LMArena also provides more detailed **category labels** inside the columns `is_code` and nested `category_tag` column.
We have already extracted the following boolean Series for you. Feel free to refer to these columns when you are analyzing your clusters for Question 3c.

**Task:** 
1. Make a bar chart showing these proportion for each category (i.e. the fraction of battles where the category is True)
2. Using `get_pairwise_win_fraction_plot`, compute the pairwise win rate for each category and use the `plot_category_rank_heatmap` to visualize the results. Ensure that your data is in *tidy format*, containing columns `model`, `category`, and `win_rate`. You should have each of the categories above as well as an 'overall' category which is the `pairwise_win_rate` you computed previously.

In [ ]:
def plot_category_rank_heatmap(df: pd.DataFrame) -> None:
    """
    Plot a heatmap of model ranks by category.
    """
    assert "overall" in df["category"].unique(), "'overall' was not found as a category in your dataframe"
    rank_table = df.pivot(index="model", columns="category", values="rank")

    if "overall" in rank_table.columns:
        cols = ["overall"] + [c for c in rank_table.columns if c != "overall"]
        rank_table = rank_table[cols]
    # sort models by overall rank
    rank_table = rank_table.sort_values("overall", ascending=True)
    px.imshow(
        rank_table,
        text_auto=True,
        color_continuous_scale="Viridis_r",
        labels=dict(x="Category", y="Model", color="Rank (1=best)"),
        zmin=1, zmax=rank_table.max().max(),
        aspect="auto"
    ).update_layout(
        title="Overall and Per-Category Ranks",
        width=950,
        height=400 + 12 * len(rank_table),
        xaxis_side="top"
    ).show()

In [ ]:
# LMArena also provides more detailed category labels inside the columns `is_code`, `is_refusal`,
# and the nested `category_tag` column.
# We have already extracted the following boolean Series for you:

# GIVEN (do not modify)
expected_creative = eng_battles_no_ties['category_tag'].apply(lambda x: x['criteria_v0.1']['creativity'])
expected_tech = eng_battles_no_ties['category_tag'].apply(lambda x: x['criteria_v0.1']['technical_accuracy'])
expected_if = eng_battles_no_ties['category_tag'].apply(lambda x: x['if_v0.1']['if'])
expected_math = eng_battles_no_ties['category_tag'].apply(lambda x: x['math_v0.1']['math'])
expected_code = (eng_battles_no_ties['is_code'] == True)

# Task:
# 1) Make a bar chart showing these proportion for each category (i.e. the fraction of battles where the category is True)
# 2) Compute the pairwise win rate for each category and use the plot_category_rank_heatmap to visualize the results

# TODO: plot a bar chart of the proportions
...

# TODO: compute the pairwise win rate for each category and use the plot_category_rank_heatmap to visualize the results
# Ensure that your data is tidy format (i.e. has the columns model, category, and win_rate). We have added the category column to the pairwise_win_rate dataframe below.

pairwise_win_rate['category'] = 'overall' 
for category in category_dataframes:
    ...

rank_dataframes = ...
plot_category_rank_heatmap(rank_dataframes)


In [ ]:
grader.check("q3b")

<!-- BEGIN QUESTION -->

You might notice that the leaderboards can change quite a bit! This is because different model developers often put more emphasis on ceratin tasks in training their LLMs to better cater to their audience. Many of these models are also limited by the amount of data and compute available, which can further force specialization as some tasks are much harder to learn (esoecially if the model is on the smaller side). 

We will explore these ideas more in part 2 of the homework.

## **Question 3c**

The category labels provide a clean way to divide problems but there are likely other ways to group prompts which can reveal other common use cases. 

We'll now dig deeper by discovering prompt topics via K-Means clustering on the **prompt text**. Since we are interested in seeing what kinda of questions people are asking, use the **no-top-prompts** subset from earlier (`eng_battles_no_ties_no_top_prompts`) to reduce noise. We have sampled 8,000 battles for reasonable runtime and provided a helper function for clustering.


**Task:**

1. We already provided you eng_battles_no_ties_no_top_prompts with the 8000 samples
2.  Build TF-IDF features X from eng_battles_no_ties_no_top_prompts["prompt"] (default max_features=500)
3.  Sweep K over [4, 6, 8, 10, 12] collecting times and inertias
4.  Plot runtime vs K and elbow (inertia vs K)
5.  Choose a best_K (e.g., 8) from elbow
6. Assign labels to eng_battles_no_ties_no_top_prompts["cluster"]
7. Visualize the clusters using a 2D projection (e.g., SVD - see [TruncatedSVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)), with points colored by their cluster.
8.  Save the clustered df to clustered_prompts_no_top_prompts.csv

#### More Gradio Visualizations!

We have provided another [gradio app](https://huggingface.co/spaces/BerkeleyML/Arena-Cluster-Viewer) for yall to view prompts in each cluster, but feel free to modify it if you want to view different subsets of conversations!

<div style="text-align: center;">
  <img src="https://i.imgur.com/r9b0xxQ.png" alt="GradioApp" style="display: block; margin-left: auto; margin-right: auto; width: 60%;">
</div>

Try it yourself:  You can use Gradio to build an app and easily share your visualization—just like in the staff example! This is a great way to help others conveniently view the clusters you’ve discovered (and delve into the wonderful world of UI vibe coding).

### Helper Functions

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

# Since we're focusing on prompts again, use the no-power-user subset from earlier:
eng_battles_no_ties_no_top_prompts = eng_battles_no_ties_no_top_prompts.sample(n=8000, random_state=42)

texts = np.array(eng_battles_no_ties_no_top_prompts["prompt"])

# Fit the TfidfVectorizer and transform texts in one step, storing the result in X
# (feel free to experiment with max_features later in 2c follow-up)
vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1,1), min_df=5)
X = vectorizer.fit_transform(texts)

In [ ]:
# General KMeans for any embedding
def kmeans_cluster_prompts(features: np.ndarray, prompts: np.ndarray, k: int, random_state: int = 42):
    """
    Perform k-means clustering on features and return:
      - dataframe with prompts and their cluster assignments
      - model inertia (float)
      - elapsed runtime (seconds)
    """
    t0 = time.perf_counter()
    km = KMeans(n_clusters=k, random_state=random_state, n_init=10)
    cluster_labels = km.fit_predict(features)
    elapsed = time.perf_counter() - t0
    df = pd.DataFrame({"prompt": prompts, "cluster": cluster_labels})
    return df, km.inertia_, elapsed


# Example usage of the function (updated to unpack df, inertia, elapsed)
clustered_prompts_df, inertia, elapsed = kmeans_cluster_prompts(X, texts, k=5)

print("Clustered prompts dataframe shape:", clustered_prompts_df.shape)
print("Inertia:", inertia)
print("Elapsed time (s):", elapsed)

print("\nFirst few rows:")
print(clustered_prompts_df.head())

print("\nCluster distribution:")
print(clustered_prompts_df['cluster'].value_counts().sort_index())

# Save to CSV
clustered_prompts_df.to_csv('clustered_prompts_no_top_prompts.csv', index=False)
print("\nSaved to clustered_prompts_no_top_prompts.csv")

In [ ]:
# Since we're focusing on the prompt text and removing power-user bias,
# use the no top prompts subset `eng_no_top_prompts_battles`:

# 1) We already provided you eng_battles_sample with the 8000 samples
# 2) Build TF-IDF features X from eng_battles_sample["prompt"] (default max_features=500)
# 3) You can use kmeans_cluster_prompts(features, prompts, k, random_state) to return:
#       - DataFrame ['prompt','cluster'], inertia (float), elapsed seconds (float)
# 4) Sweep K over [4, 6, 8, 10, 12] collecting times and inertias
# 5) Plot runtime vs K and elbow (inertia vs K)
# 6) Choose a best_K (e.g., 8) from elbow
# 7) Assign labels to eng_battles_sample["cluster"]
# 8) Visualize with 2D projection (ex: using SVD) colored by cluster
# 9) Save the clustered df to clustered_prompts_no_top_prompts.csv

...
# Save to CSV
out_path = f"clustered_prompts_no_top_prompts_k{best_K}.csv"
YOUR_DF.to_csv(out_path, index=False)
print(f"Saved -> {out_path}")


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## **Question 3d**

We’ll now move from training to interpretation. Using the cluster labels from your selected K, briefly characterize what at least one distinct cluster seems to contain, or explain why patterns are unclear and how you might improve them.

For more clear visulization, we suggest you to explore gradio by working with the clustered_prompts_no_top_prompts.csv that was mentioned above.

**Task: Answer the questions below**

```otter
In 2–3 sentences, use the labels from your clustering to do the following:
1. Briefly explain the type(s) of question you see in one of the distinct/similar clusters.
2. If you don’t see a clear pattern, describe likely limitations and potential improvements we can make.
```

```otter
YOUR ANSWER:
```

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## **Question 3e**

In this homework we have seen how different prompts can lead to leaderboard changes, now its your time to find other prompt types that cause leaderboard shift!

**Task:** Find a specific subset of battles in `eng_battles_no_ties_no_top_prompts` that, when removed, causes the **top 5 models to change rankings**. You must create a clear, simple rule for selecting which battles to remove.

To get points for this question, you must give the following deliverables:
1. Provide your code for selecting the battles to be removed and save your final dataframe with your subset of battles filtered out to a variable called `filtered_battles` 
2. Produce the leaderboards for the overall leaderboard and the leaderboard of your split using `plot_category_rank_heatmap` to show rank change
2. Write 1-3 sentences explaining your solution and why you suspect the leaderboard has shifted. Your solution should have a clear "rule" of how you selected your prompts (e.g. I removed all the prompts which contained the word `banana').

-------

You are free to use any of the columns in `eng_battles_no_ties_no_top_prompts` or create a new column with your custom "rule" (e.g. "contains the word banana"). For your solution to be valid it **must** adhere to the following properties:
* The subset of prompts you remove **must be <=20% of the total prompts** in `eng_battles_no_ties_no_top_prompts` and must follow a clear rule. For instance, randomly removing 20% of prompts from a prompt cluster is invalid because you are subsampling - you need to define a group where removing all of the prompts removes less than 20% of the overall prompts. Similarly, having a casade of several unrelated rules is also invalid. 
* Each model in `selected_models` **must appear in at least 100 battles** in your final `filtered_battles` dataframe
* You cannot simply replicate work from 3a or 3b. For example, you may not remove an entire question category or part of one, nor just delete all “hi” and “hello” prompts. 
* Your solution must select battles **based on the inputs (either the prompts or the users), not the models**. For example, removing the `is_refusal` category is an invalid solution as refusal is a property of the model response and ot the prompt itself. 

This intentionally very open ended so don't overthink it! As long as you can clearly describe what types of prompts you removed and why their removal might result in a ranking change you will get full points. 

In [ ]:
def filter_out_battles(eng_battles_no_ties_no_top_prompts: pd.DataFrame) -> pd.DataFrame:
    """
    Selects and removes a subset of battles containing a specific type of input.

    Args:
        eng_battles_no_ties_no_top_prompts (pd.DataFrame): DataFrame containing battles. 
            Must include a 'judge_hash' column.

    Returns:
        pd.DataFrame: Filtered DataFrame with <=20% of the total prompts removed.
    """
    ...
    return filtered_battles

filtered_battles = filter_out_battles(eng_battles_no_ties_no_top_prompts)
custom_leaderboard, _ = get_pairwise_win_fraction_plot(filtered_battles)
custom_leaderboard["category"] = "custom"
plot_category_rank_heatmap(pd.concat([custom_leaderboard, pairwise_win_rate]))

In [ ]:
grader.check("q3e")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Free response:** Write 1-3 sentences explaining your solution and why you suspect the leaderboard has shifted. Your solution should have a clear "rule" of how you selected your prompts (e.g. I removed all the prompts which contained the word `banana').

```otter
YOUR ANSWER:
```

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
## Use this cell if you are running the notebook in Google Colab to install the necessary dependencies, this may take a few minutes
if IS_COLAB:
    !apt-get install -y texlive texlive-xetex pandoc


In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)